In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Dropout, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.src.utils import pad_sequences
from sklearn.model_selection import train_test_split
from custom_preprocessor import preprocess
import json

/var/folders/lx/3tpp7mdx68918pcn6277_wtw0000gn/T/ipykernel_62687/1599806223.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Get data
Pandas is used to get the data from the project directory.
The data is a .csv file, that has been created by book texts and prepared to be listed in the file word-by-word.

In [2]:
df = pd.read_csv('./data/dataset_uralic.csv')
df = df.sample(frac=1.0, random_state=12)
df.head()

,Word,Language
574,jaan su paris jaan su paris jaan su paris jaan,Estonian
4267,kasakkoja kohdeltiin kasakkoja kohdeltiin,Finnish
4,margus onnelikus arewuses margus onnelikus,Estonian
2244,vissza az ujonnan vissza az ujonnan vissza az,Hungarian
2982,egeszsegtelen es a egeszsegtelen es a,Hungarian


# Prepare data
Divide the data into X and y. X is input and y is the desired output.
One-Hot Encode it, so the language is categorical. This is done with Pandas.

In [3]:
# Getting X - the input values
X = df.iloc[:, :-1]
X.loc[:, 'Word'] = X['Word'].astype(str)
tokenizer = Tokenizer(char_level=True) # Used for turning the words into numbers
tokenizer.fit_on_texts(X['Word'].tolist()) # Giving Tokenizer the data it needs to learn how to encode the words
X_sequences = tokenizer.texts_to_sequences(X['Word'].tolist()) # Now the words are encoded into sequences
input_dim = max(len(seq) for seq in X_sequences)
print(f"Input dimension: {input_dim}")
X = pad_sequences(
    X_sequences, 
    maxlen=input_dim, 
    padding='post'
) # Making all input values be of same length
print(f"Rows: {len(X)}")

# Getting y - the output values
y_df = pd.get_dummies(df.iloc[:, -1:])
y = y_df.values
print(f"Amount of languages to identify: {len(y[1])}")

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

Input dimension: 48
Rows: 4785
Amount of languages to identify: 3


In [4]:
model = Sequential()
model.add(Conv1D(input_dim, 2, activation='relu'))  # Convolutional layer
tf.keras.layers.SimpleRNN(8, activation='relu')
model.add(LSTM(32, return_sequences=True))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(len(y[1]), activation='softmax'))
adam = optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [5]:
history = model.fit(X_train.astype('float32'), y_train.astype('float32'), validation_data=(X_val, y_val), epochs=300, batch_size=2048,verbose=1)

Epoch 1/300
2/2 [==============================] - 1s 304ms/step - loss: 1.8370 - accuracy: 0.3331 - val_loss: 1.4819 - val_accuracy: 0.3239
Epoch 2/300
2/2 [==============================] - 0s 148ms/step - loss: 1.5121 - accuracy: 0.3278 - val_loss: 1.0842 - val_accuracy: 0.3699
Epoch 3/300
2/2 [==============================] - 0s 155ms/step - loss: 1.1692 - accuracy: 0.3391 - val_loss: 1.2391 - val_accuracy: 0.3323
Epoch 4/300
2/2 [==============================] - 0s 158ms/step - loss: 1.1806 - accuracy: 0.3542 - val_loss: 1.0622 - val_accuracy: 0.4577
Epoch 5/300
2/2 [==============================] - 0s 161ms/step - loss: 1.0805 - accuracy: 0.4198 - val_loss: 1.1166 - val_accuracy: 0.3678
Epoch 6/300
2/2 [==============================] - 0s 159ms/step - loss: 1.1248 - accuracy: 0.3939 - val_loss: 1.0737 - val_accuracy: 0.5225
Epoch 7/300
2/2 [==============================] - 0s 156ms/step - loss: 1.0636 - accuracy: 0.4770 - val_loss: 1.0412 - val_accuracy: 0.4566
Epoch 8/300
2

KeyboardInterrupt: 

# Loss evaluation

Seeing how the model's loss changes by the epoch

In [ ]:
%matplotlib inline
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Loss")
plt.legend(['Train', 'Val'], loc='upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')

# Loss evaluation

Seeing how the model's loss changes by the epoch

In [ ]:
%matplotlib inline
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Accuracy")
plt.legend(['Train', 'Val'], loc='upper right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

# Prediction

In [ ]:
prediction_word = "hej"
print(f"Text before preprocessing: {prediction_word}")
prediction_word = preprocess(prediction_word)
print(f"Text after preprocessing: {prediction_word}")
prediction_word = tokenizer.texts_to_sequences([prediction_word])
prediction_word = pad_sequences(prediction_word, maxlen=input_dim)
prediction = model.predict([prediction_word])
output_probabilities = np.array(prediction)
result = {}
class_labels = np.unique(y_df.columns)
for index, prediction in enumerate(output_probabilities[0]):
    result.update({class_labels[index]: round(prediction * 100, 1)})

# Print keys and values of the result dictionary
for key, value in result.items():
    print(f"{key}: {value}")

# Save model
Here a model can be loaded

In [ ]:
model.save('./models/model_uralic.keras')

# Load model
Here a model can be loaded

In [ ]:
model = load_model('./models/model_uralic.keras')
tokenizer_json = tokenizer.to_json()
with open('./tokenizer_configs/tokenizer_config_uralic.json', 'w') as config_file:
    config_file.write(json.dumps(tokenizer_json, ensure_ascii=False))